In [ ]:
# Hamiltonian matrix
function minstate(tag::Int, t::Array{Int64,2})
    dim = size(t,1)
    ms = 0
    for i = 1: dim
        if t[i, tag + 1] != 0
            ms = i - 1
            break
        end
    end
    return ms
end

function findstate(tag::Int, mb::Array{Int,1}, t1::Array{Int64,2}, t2::Array{Int64,2}, N1::Int, N2::Int)
    #=Find the reference state of an arbitrary state.
       Inputs: tag :tag of the input state
               mb: list of momentum basis
               t1, t2: Translation matrix
       Outputs: ntag: tag of the corresponding reference state
                l : translation steps
                ntag and l will return negative values if tag is not in mbasis.=#     
    l1 = -1
    l2 = -1
    
    ms1 = minstate(tag, t1)
    l1 =  N1 - t1[ms1 + 1, tag + 1]
    if ms1 in mb
        l2 = 0
        return ms1, l1, l2
    else
        ms2 = minstate(ms1, t2)
        if ms2 in mb
            l2 = N2 - t2[ms2 + 1, ms1 + 1]
            return ms2, l1, l2
        else
            ms2 = -1
            l2 = -1
            l1 = -1
            return ms2, l1, l2
        end
    end
end

function findindex(tag::Int, mb::Array{Int,1})
    # find the index of a tag in mb
    # binary search
    dim = length(mb)
    low = 1
    high = dim   
    while low < high
        mid = low + div(high - low, 2)
        if tag < mb[mid]
            high = mid - 1
        elseif tag > mb[mid]
            low = mid + 1
        elseif tag == mb[mid]
            return mid
        end
    end
end
                           
function KitaevH(J::Array{Float64,1}, N1::Int, N2::Int, k1::Int, k2::Int)
    #=Kitaev Hamiltonian on N1 * N2 * 2 lattice for a given k (with translation symmetry)
       N1: number of unit cells in n1-direction, column number
       N2: number of unit cells in n1-direction, row number
    
       Inputes: 
           N1, N2: width and length of the rhomboid cluster
           k1, k2: momentund
        
       Outputs: The Non-zero Hamiltonian element H[i] is in the 
                position (row[i], col[i])
                H, row, col : type: list
        =#
    mb, ra = MomBasis(k1, k2, N1, N2)
   
    dim = length(mb)
    f1 = -1im * 2 * pi / N1
    f2 = -1im * 2 * pi / N2
    
    base_dic = Dict{Int,Int}()
    for i = 1: dim
        base_dic[mb[i]] = i
    end
    
    data = Complex[]
    row = Int[]
    col = Int[]
    
    for tag in mb
        ind  = base_dic[tag]
        temp = Dict{Int, Complex}() # A temporay dict
        
        for r = 0: N2-1
            for c = 0: N1-1
                ini = index(N1, N2, c, r, 0)
                
                # Sx-Sx
                next = index(N1, N2, c, r, 1)
                
                ntag = flip(tag, ini, next, N1, N2)
                rtag, l1, l2 = findstate(ntag, mb, N1, N2)
                if rtag > 0
                    nind = base_dic[rtag]
                    phase = exp(f1*k1*l1 + f2*k2*2) * sqrt(ra[ind]/ra[nind])
                    if nind in keys(temp)
                        temp[nind] += -J[1]/4 * phase
                    else
                        temp[nind] = -J[1]/4 * phase
                    end
                end
                                
                # Sy-Sy
                next = index(N1, N2, c - 1, r, 1)
                    
                ntag = flip(tag, ini, next, N1, N2)
                rtag, l = findstate(ntag, mb, N1, N2)
    
                if rtag > 0
                    nind = base_dic[rtag]
                    phase = exp(f1*k1*l1 + f2*k2*l2) * sqrt(ra[ind]/ra[nind])
                    
                    if nind in keys(temp)
                        temp[nind] += J[2]*(bits(tag, ini, N1, N2)-0.5)*(bits(tag, next, N1, N2)-0.5)*phase           
                    else
                        temp[nind] = J[2]*(bits(tag, ini, N1, N2)-0.5)*(bits(tag, next, N1, N2)-0.5)*phase
                    end
                end
                                    
                # Sz-Sz
                next = index(N1, N2, c, r - 1, 1)
                    
                if ind in keys(temp)
                    temp[ind] += - J[3] * (bits(tag, ini, N1, N2)-0.5)*(bits(tag, next, N1, N2)-0.5)
                else
                    temp[ind] = - J[3] *(bits(tag, ini, N1, N2)-0.5)*(bits(tag, next, N1, N2)-0.5)
                end
            end
        end
                        
        #odTemp = sort!(collect(keys(temp)))
        for sf in keys(temp)
            append!(col,[ind])
            append!(row,[sf])
            append!(data,[temp[sf]])
        end
    end
    return col, row, data
end

function KitaevH_opt(J::Array{Float64,1}, N1::Int, N2::Int, k1::Int, k2::Int)
    #=Kitaev Hamiltonian on N1 * N2 * 2 lattice for a given k (with translation symmetry)
       N1: number of unit cells in n1-direction, column number
       N2: number of unit cells in n1-direction, row number
    
       Inputes: 
           N1, N2: width and length of the rhomboid cluster
           k1, k2: momentund
        
       Outputs: The Non-zero Hamiltonian element H[i] is in the 
                position (row[i], col[i])
                H, row, col : type: list
        =#
    mb, ra, t1, t2 = MomBasis_opt(k1, k2, N1, N2)

    dim = length(mb)
    f1 = -1im * 2 * pi / N1
    f2 = -1im * 2 * pi / N2
    

    H = spzeros(Complex, dim, dim)

    for tag in mb
        i = findindex(tag, mb)
        for r = 0: N2-1
            for c = 0: N1-1
                si = index(N1, N2, c, r, 0) # start index
                
                # Sx-Sx
                ni = index(N1, N2, c, r, 1) # next index
                ntag = flip(tag, si, ni, N1, N2)
                rtag, l1, l2 = findstate(ntag, mb, t1, t2, N1, N2)
                if rtag > 0
                    j = findindex(rtag, mb)
                    phase = exp(f1*k1*l1 + f2*k2*l2) * sqrt(ra[i]/ra[j])
                    H[j, i] += -J[1]/4 * phase
                end
                                
                # Sy-Sy
                ni = index(N1, N2, c - 1, r, 1)
                ntag = flip(tag, si, ni, N1, N2)
                rtag, l1, l2 = findstate(ntag, mb, t1, t2, N1, N2)
                if rtag > 0
                    j = findindex(rtag, mb)
                    phase = exp(f1*k1*l1 + f2*k2*l2) * sqrt(ra[i]/ra[j])
                    H[j, i] += J[2]*(bits(tag, si, N1, N2)-0.5)*(bits(tag, ni, N1, N2)-0.5)*phase
                end
                                    
                # Sz-Sz
                ni = index(N1, N2, c, r - 1, 1)
                H[i, i] += - J[3] * (bits(tag, si, N1, N2)-0.5)*(bits(tag, ni, N1, N2)-0.5)
            end
        end
    end
    return H
end